<a href="https://colab.research.google.com/github/nonyeezeh/Research-Project-Code/blob/main/LBN_Dense_AIC_6_Nodes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [1]:
pip install pgmpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 15.4 MB/s eta 0:00:00


In [3]:
import numpy as np
import pandas as pd
from pgmpy.estimators import HillClimbSearch, AICScore, MaximumLikelihoodEstimator
from pgmpy.models import BayesianNetwork
from pgmpy.estimators import BayesianEstimator
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from tabulate import tabulate

# Bayesian Network Data Generation 500, ..., 20000 Samples (dense)

In [4]:
# Function to generate CPDs for the new structure
def generate_cpds():
    # Generate random probabilities for IR
    ir_probs = np.random.rand(3)
    ir_probs /= ir_probs.sum()  # Normalize to make it a valid probability distribution

    # Generate random probabilities for IRT given IR
    irt_given_ir_probs = np.random.rand(3, 3)
    irt_given_ir_probs /= irt_given_ir_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for EI given IR and IRT
    ei_given_ir_irt_probs = np.random.rand(3, 3, 3)
    ei_given_ir_irt_probs /= ei_given_ir_irt_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for GEO
    geo_probs = np.random.rand(3)
    geo_probs /= geo_probs.sum()

    # Generate random probabilities for UE given GEO
    ue_given_geo_probs = np.random.rand(3, 3)
    ue_given_geo_probs /= ue_given_geo_probs.sum(axis=0, keepdims=True)

    # Generate random probabilities for SP given IR, EI, and UE
    sp_probs = np.random.rand(3, 3, 3, 3)
    sp_probs /= sp_probs.sum(axis=0, keepdims=True)

    return ir_probs, irt_given_ir_probs, ei_given_ir_irt_probs, geo_probs, ue_given_geo_probs, sp_probs

# Function to generate and save samples with the new nodes
def generate_and_save_samples(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, sp_probs, sample_size, filename):
    output_data = []

    # Generate `sample_size` random samples
    for _ in range(sample_size):
        # Sample IR
        ir_state_idx = np.random.choice(3, p=ir_probs)
        ir_state = ['low', 'medium', 'high'][ir_state_idx]
        ir_prob = ir_probs[ir_state_idx]

        # Sample IRT given IR
        irt_probs_given_ir = irt_probs[:, ir_state_idx]
        irt_state_idx = np.random.choice(3, p=irt_probs_given_ir)
        irt_state = ['decreasing', 'steady', 'increasing'][irt_state_idx]
        irt_prob = irt_probs_given_ir[irt_state_idx]

        # Sample EI given IR and IRT
        ei_probs_given_ir_irt = ei_probs[:, ir_state_idx, irt_state_idx]
        ei_state_idx = np.random.choice(3, p=ei_probs_given_ir_irt)
        ei_state = ['poor', 'average', 'good'][ei_state_idx]
        ei_prob = ei_probs_given_ir_irt[ei_state_idx]

        # Sample GEO
        geo_state_idx = np.random.choice(3, p=geo_probs)
        geo_state = ['recession', 'stable', 'growth'][geo_state_idx]
        geo_prob = geo_probs[geo_state_idx]

        # Sample UE given GEO
        ue_probs_given_geo = ue_probs[:, geo_state_idx]
        ue_state_idx = np.random.choice(3, p=ue_probs_given_geo)
        ue_state = ['high', 'medium', 'low'][ue_state_idx]
        ue_prob = ue_probs_given_geo[ue_state_idx]

        # Sample SP given IR, EI, and UE
        sp_probs_given_ir_ei_ue = sp_probs[:, ir_state_idx, ei_state_idx, ue_state_idx]
        sp_state_idx = np.random.choice(3, p=sp_probs_given_ir_ei_ue)
        sp_state = ['decrease', 'stable', 'increase'][sp_state_idx]
        sp_prob = sp_probs_given_ir_ei_ue[sp_state_idx]

        # Append sample data to output list including probabilities for all nodes
        output_data.append({
            'IR_State': ir_state,
            'IR_Prob': f'{ir_prob:.4f}',
            'IRT_State': irt_state,
            'IRT_Prob': f'{irt_prob:.4f}',
            'EI_State': ei_state,
            'EI_Prob': f'{ei_prob:.4f}',
            'GEO_State': geo_state,
            'GEO_Prob': f'{geo_prob:.4f}',
            'UE_State': ue_state,
            'UE_Prob': f'{ue_prob:.4f}',
            'SP_Probabilities (decrease, stable, increase)': ', '.join([f'{prob:.4f}' for prob in sp_probs_given_ir_ei_ue]),
            'Chosen_SP_State': sp_state,
            'Chosen_SP_Probability': f'{sp_prob:.4f}'
        })

    # Create a DataFrame from the output data
    output_df = pd.DataFrame(output_data)

    # Save the output DataFrame to a CSV file
    output_df.to_csv(filename, index=False)

    # Print the first few rows for visual confirmation
    print(f"\nSample size: {sample_size} - First few rows of generated samples:\n")
    print(tabulate(output_df.head(), headers='keys', tablefmt='grid'))

# Generate and save samples for sample sizes
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

for size in sample_sizes:
    ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, sp_probs = generate_cpds()
    generate_and_save_samples(ir_probs, irt_probs, ei_probs, geo_probs, ue_probs, sp_probs, size, f'combined_probabilities_{size}.csv')

# Notify the user that the process is done
print("\nGeneration and saving of individual samples complete for all sample sizes!")


Sample size: 50 - First few rows of generated samples:

+----+------------+-----------+-------------+------------+------------+-----------+-------------+------------+------------+-----------+-------------------------------------------------+-------------------+-------------------------+
|    | IR_State   |   IR_Prob | IRT_State   |   IRT_Prob | EI_State   |   EI_Prob | GEO_State   |   GEO_Prob | UE_State   |   UE_Prob | SP_Probabilities (decrease, stable, increase)   | Chosen_SP_State   |   Chosen_SP_Probability |
+====+============+===========+=============+============+============+===========+=============+============+============+===========+=================================================+===================+=========================+
|  0 | low        |    0.3961 | increasing  |     0.2862 | poor       |    0.4848 | stable      |     0.4432 | low        |    0.11   | 0.2917, 0.6818, 0.0264                          | stable            |                  0.6818 |
+----+---------

# LBN Dense AIC & Entropy

In [5]:
# Sample sizes to loop through
sample_sizes = [50, 100, 150, 200, 250, 300, 350, 400, 450, 500, 550, 600, 650, 700, 750, 800, 850, 900, 950, 1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000, 11000, 12000, 13000, 14000, 15000, 16000, 17000, 18000, 19000, 20000]

# Initialize list to store K-L divergence and standard deviation values for each sample size
results = []

# Small smoothing value to avoid zero probabilities
epsilon = 1e-10

# Loop through each sample size
for sample_size in sample_sizes:
    print(f"\nProcessing sample size: {sample_size}")

    # Load the dataset for the current sample size
    data_file = f'combined_probabilities_{sample_size}.csv'
    df = pd.read_csv(data_file)

    # Manually encode categorical variables for IR, IRT, EI, GEO, UE, and SP
    ir_map = {'low': 0, 'medium': 1, 'high': 2}
    irt_map = {'decreasing': 0, 'steady': 1, 'increasing': 2}
    ei_map = {'poor': 0, 'average': 1, 'good': 2}
    geo_map = {'recession': 0, 'stable': 1, 'growth': 2}
    ue_map = {'high': 0, 'medium': 1, 'low': 2}
    sp_map = {'decrease': 0, 'stable': 1, 'increase': 2}

    df['IR_encoded'] = df['IR_State'].map(ir_map)
    df['IRT_encoded'] = df['IRT_State'].map(irt_map)
    df['EI_encoded'] = df['EI_State'].map(ei_map)
    df['GEO_encoded'] = df['GEO_State'].map(geo_map)
    df['UE_encoded'] = df['UE_State'].map(ue_map)
    df['SP_encoded'] = df['Chosen_SP_State'].map(sp_map)

    # Split the data into training, validation, and test sets
    X = df[['IR_encoded', 'IRT_encoded', 'EI_encoded', 'GEO_encoded', 'UE_encoded']]
    y = df['SP_encoded']

    X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)
    X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42, shuffle=True)

    # Concatenate X and y to form the training set for learning the BN structure
    df_train = pd.concat([X_train, y_train], axis=1)

    # Enforce priors (fixed edges)
    priors = [
        ('IR_encoded', 'SP_encoded'),
        ('IRT_encoded', 'IR_encoded'),
        ('EI_encoded', 'SP_encoded'),
        ('UE_encoded', 'SP_encoded'),
        ('GEO_encoded', 'UE_encoded')
    ]

    # Perform Hill Climb Search with priors
    hc = HillClimbSearch(df_train)
    best_dag = hc.estimate(scoring_method=AICScore(df_train), fixed_edges=priors)

    # Initialize BayesianNetwork and print edges
    best_model = BayesianNetwork(best_dag.edges())
    print("Learned structure (edges):", best_model.edges())

    if len(best_model.edges()) == 0:
        print("No edges learned. Skipping to next sample size.")
        continue

    # Try fitting parameters using BayesianEstimator
    try:
        best_model.fit(df_train, estimator=BayesianEstimator, prior_type="BDeu", equivalent_sample_size=5)
        print("Model fitted successfully.")
    except Exception as e:
        print("Error during fitting:", str(e))
        continue

    # Check model validity
    if not best_model.check_model():
        print("Model check failed.")
        continue

    # Predict on test data and calculate K-L divergence
    try:
        inference = best_model.predict(X_test)
        predicted_probabilities = inference['SP_encoded']

        ground_truth_probabilities = y_test.value_counts(normalize=True).sort_index()
        predicted_probabilities = predicted_probabilities.value_counts(normalize=True).sort_index()

        # Reindex both distributions and add smoothing
        all_categories = sorted(set(ground_truth_probabilities.index).union(set(predicted_probabilities.index)))
        ground_truth_probabilities = ground_truth_probabilities.reindex(all_categories, fill_value=epsilon)
        predicted_probabilities = predicted_probabilities.reindex(all_categories, fill_value=epsilon)

        kl_divergence = entropy(pk=ground_truth_probabilities, qk=predicted_probabilities)
        std_dev = np.std(predicted_probabilities - ground_truth_probabilities)

        results.append({
            'Sample_Size': sample_size,
            'K-L_Divergence': kl_divergence,
            'Standard_Deviation': std_dev
        })

        print(f"K-L Divergence: {kl_divergence:.4f}")
        print(f"Standard Deviation: {std_dev:.4f}")

    except Exception as e:
        print("Error during prediction:", str(e))

# Save the K-L divergence results
results_df = pd.DataFrame(results)
results_df.to_csv('kl_std_aic_with_new_nodes.csv', index=False)

print("\nAll results have been saved.")


Processing sample size: 50


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IR_encoded', 'GEO_encoded'), ('GEO_encoded', 'UE_encoded'), ('IRT_encoded', 'IR_encoded'), ('EI_encoded', 'SP_encoded'), ('UE_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/7 [00:00<?, ?it/s]

K-L Divergence: 5.2072
Standard Deviation: 0.2041

Processing sample size: 100


...

K-L Divergence: 0.0273
Standard Deviation: 0.0738

Processing sample size: 20000


  0%|          | 0/1000000 [00:00<?, ?it/s]

Learned structure (edges): [('IR_encoded', 'SP_encoded'), ('IRT_encoded', 'IR_encoded'), ('IRT_encoded', 'EI_encoded'), ('IRT_encoded', 'GEO_encoded'), ('EI_encoded', 'SP_encoded'), ('EI_encoded', 'IR_encoded'), ('EI_encoded', 'GEO_encoded'), ('GEO_encoded', 'UE_encoded'), ('UE_encoded', 'SP_encoded')]
Model fitted successfully.


  0%|          | 0/198 [00:00<?, ?it/s]

K-L Divergence: 0.3430
Standard Deviation: 0.2619

All results have been saved.
